In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [6]:
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
#df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

# PanelOLS

In [4]:
!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

In [7]:
'''df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
authority_code = pd.Categorical(df_reg_exp.authority_code)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = authority_code
df_reg_exp.head()   
#exog_var = ['fpsb_auction','year','auth_anno']
#exog = sm.add_constant(df_reg_exp[exog_var])
#mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = False)  
#fe_res = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})  
#print(fe_res) '''

auth_anno  fpsb_auction  n_bidders   discount  \
authority_code year                                                    
3090272.0      2000.0  3090272.0           0.0       85.0  21.160000   
               2000.0  3090272.0           0.0       55.0  14.570000   
               2000.0  3090272.0           0.0       41.0  20.889999   
               2000.0  3090272.0           0.0       67.0  15.850000   
               2000.0  3090272.0           0.0       51.0  26.700001   

                       reserve_price work_category  complex_work  delay_ratio  \
authority_code year                                                             
3090272.0      2000.0       655828.0          OS24           1.0   116.867470   
               2000.0       640778.0          OG03           1.0   153.556488   
               2000.0       325368.0          OG01           2.0   206.722687   
               2000.0      1851498.0          OG03           1.0          NaN   
               2000.0       381504.0          OS21           2.0   103.355705   

                       overrun_ratio  days_to_award  ...  \
authority_code year                                  ...   
3090272.0      2000.0      11.712280          213.0  ...   
               2000.0      14.264524          216.0  ...   
               2000.0            NaN           44.0  ...   
               2000.0      11.934337           15.0  ...   
               2000.0            NaN          106.0  ...   

                       ctrl_pop_exp_turin_pr_sample  OG03_dummy  OG01_dummy  \
authority_code year                                                           
3090272.0      2000.0                           1.0           0           0   
               2000.0                           1.0           1           0   
               2000.0                           1.0           0           1   
               2000.0                           1.0           1           0   
               2000.0                           1.0           0           0   

                       OG_rest_dummy  OG_dummy  OS_dummy  trend  trend_treat  \
authority_code year                                                            
3090272.0      2000.0              1         0         1    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              1         0         1    1.0          1.0   

                       trend_control    year  
authority_code year                           
3090272.0      2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  

[5 rows x 38 columns]

# 1. Col1

In [20]:
#table2 = co
#outcomes "discount delay_ratio overrun_ratio days_to_award"
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
outcomes = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

authority_code  year  
3.0             2005.0    15.200000
                2005.0    14.720000
                2002.0    17.879999
                2005.0    21.830000
                2005.0    19.180000
Name: discount, dtype: float64
authority_code  year  
3.0             2005.0    40.547947
                2005.0    17.362270
                2002.0          NaN
                2005.0     0.000000
                2005.0    38.333332
Name: delay_ratio, dtype: float32
authority_code  year  
3.0             2005.0    4.843400
                2005.0    4.794236
                2002.0   -0.004622
                2005.0   -3.869733
                2005.0    4.913357
Name: overrun_ratio, dtype: float32
authority_code  year  
3.0             2005.0     5.0
                2005.0    50.0
                2002.0     8.0
                2005.0    62.0
                2005.0    23.0
Name: days_to_award, dtype: float32


In [31]:
#Winning discount
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.4729
Estimator:                   PanelOLS   R-squared (Between):              0.9897
No. Observations:                1262   R-squared (Within):               0.2000
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.4674
Time:                        10:45:18   Log-likelihood                   -3953.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.164
Entities:                          15   P-value                           0.0000
Avg Obs:                       84.133   Distribution:                 F(15,1240)
Min Obs:                       28.000                                           
Max Obs:                       200.00   F-statistic (robust):             74.164
                            

In [32]:
#completion time dealy
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:            delay_ratio   R-squared:                        0.0741
Estimator:                   PanelOLS   R-squared (Between):              0.9910
No. Observations:                1110   R-squared (Within):               0.0002
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.0717
Time:                        10:45:46   Log-likelihood                   -6227.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      5.8070
Entities:                          15   P-value                           0.0000
Avg Obs:                       74.000   Distribution:                 F(15,1088)
Min Obs:                       21.000                                           
Max Obs:                       167.00   F-statistic (robust):             5.8070
                            

In [33]:
#cost_overrun
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:          overrun_ratio   R-squared:                        0.0960
Estimator:                   PanelOLS   R-squared (Between):              0.9970
No. Observations:                1092   R-squared (Within):               0.0175
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.1038
Time:                        10:48:47   Log-likelihood                   -4230.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      7.5737
Entities:                          15   P-value                           0.0000
Avg Obs:                       72.800   Distribution:                 F(15,1070)
Min Obs:                       23.000                                           
Max Obs:                       162.00   F-statistic (robust):             7.5737
                            

# Col3 Ctrl_pop

In [32]:
#"discount delay_ratio overrun_ratio days_to_award" 
#Winning discount/ value checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#completion time delay/ value checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#cost overrun/ outcome is different with paper but same with stata
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#days to award the contract
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

C:\Users\thxkn\anaconda3\lib\site-packages\linearmodels\panel\model.py:1669: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

year.2001.0, year.2002.0, year.2003.0, year.2004.0, year.2005.0, year.2006.0

  AbsorbingEffectWarning,


                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.4643
Estimator:                   PanelOLS   R-squared (Between):              0.9744
No. Observations:                1275   R-squared (Within):               0.1994
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.4563
Time:                        15:33:20   Log-likelihood                   -3995.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      44.921
Entities:                          24   P-value                           0.0000
Avg Obs:                       53.125   Distribution:                 F(24,1244)
Min Obs:                       7.0000                                           
Max Obs:                       200.00   F-statistic (robust):             44.921
                            

auth_anno  fpsb_auction  n_bidders   discount  \
authority_code year                                                   
3.0            2005.0         2           0.0       25.0  15.200000   
               2005.0         2           0.0       21.0  14.720000   
               2002.0         2           0.0       72.0  17.879999   
               2005.0         2           0.0      136.0  21.830000   
               2005.0         2           0.0       46.0  19.180000   

                       reserve_price work_category  complex_work  delay_ratio  \
authority_code year                                                             
3.0            2005.0      1129754.0          OG02           3.0    40.547947   
               2005.0      1982000.0          OG01           2.0    17.362270   
               2002.0       620000.0          OG03           1.0          NaN   
               2005.0       738612.0          OG03           1.0     0.000000   
               2005.0       435225.0          OS28           2.0    38.333332   

                       overrun_ratio  days_to_award  ...  Iwork_ca_27  \
authority_code year                                  ...                
3.0            2005.0       4.843400            5.0  ...            0   
               2005.0       4.794236           50.0  ...            0   
               2002.0      -0.004622            8.0  ...            0   
               2005.0      -3.869733           62.0  ...            0   
               2005.0       4.913357           23.0  ...            0   

                       Iwork_ca_15  Iwork_ca_42  Iwork_ca_14  Iwork_ca_21  \
authority_code year                                                         
3.0            2005.0            0            0            0            0   
               2005.0            0            0            0            0   
               2002.0            0            0            0            0   
               2005.0            0            0            0            0   
               2005.0            0            0            0            0   

                       Iwork_ca_30  Iwork_ca_16  Iwork_ca_5  Iwork_ca_43  \
authority_code year                                                        
3.0            2005.0            0            0           0            0   
               2005.0            0            0           0            0   
               2002.0            0            0           0            0   
               2005.0            0            0           0            0   
               2005.0            0            0           0            0   

                         year  
authority_code year            
3.0            2005.0  2005.0  
               2005.0  2005.0  
               2002.0  2002.0  
               2005.0  2005.0  
               2005.0  2005.0  

[5 rows x 82 columns]

# Col5: ctrl_pop_exp

In [33]:
#"discount delay_ratio overrun_ratio days_to_award" 
#Winning discount/ value checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#completion time delay/ value checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#cost overrun/ outcome is different with paper but same with stata
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

#days to award the contract/ value checked
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced
df_reg_exp.head()

C:\Users\thxkn\anaconda3\lib\site-packages\linearmodels\panel\model.py:1669: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

year.2001.0, year.2002.0, year.2003.0, year.2004.0, year.2005.0, year.2006.0

  AbsorbingEffectWarning,
C:\Users\thxkn\anaconda3\lib\site-packages\linearmodels\utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.4962
Estimator:                   PanelOLS   R-squared (Between):              0.9817
No. Observations:                 880   R-squared (Within):               0.2393
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.4846
Time:                        15:37:19   Log-likelihood                   -2813.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      77.175
Entities:                          11   P-value                           0.0000
Avg Obs:                       80.000   Distribution:                  F(11,862)
Min Obs:                       28.000                                           
Max Obs:                       200.00   F-statistic (robust):             77.175
                            

auth_anno  fpsb_auction  n_bidders   discount  \
authority_code year                                                   
3.0            2005.0         2           0.0       25.0  15.200000   
               2005.0         2           0.0       21.0  14.720000   
               2002.0         2           0.0       72.0  17.879999   
               2005.0         2           0.0      136.0  21.830000   
               2005.0         2           0.0       46.0  19.180000   

                       reserve_price work_category  complex_work  delay_ratio  \
authority_code year                                                             
3.0            2005.0      1129754.0          OG02           3.0    40.547947   
               2005.0      1982000.0          OG01           2.0    17.362270   
               2002.0       620000.0          OG03           1.0          NaN   
               2005.0       738612.0          OG03           1.0     0.000000   
               2005.0       435225.0          OS28           2.0    38.333332   

                       overrun_ratio  days_to_award  ...  Iwork_ca_27  \
authority_code year                                  ...                
3.0            2005.0       4.843400            5.0  ...            0   
               2005.0       4.794236           50.0  ...            0   
               2002.0      -0.004622            8.0  ...            0   
               2005.0      -3.869733           62.0  ...            0   
               2005.0       4.913357           23.0  ...            0   

                       Iwork_ca_15  Iwork_ca_42  Iwork_ca_14  Iwork_ca_21  \
authority_code year                                                         
3.0            2005.0            0            0            0            0   
               2005.0            0            0            0            0   
               2002.0            0            0            0            0   
               2005.0            0            0            0            0   
               2005.0            0            0            0            0   

                       Iwork_ca_30  Iwork_ca_16  Iwork_ca_5  Iwork_ca_43  \
authority_code year                                                        
3.0            2005.0            0            0           0            0   
               2005.0            0            0           0            0   
               2002.0            0            0           0            0   
               2005.0            0            0           0            0   
               2005.0            0            0           0            0   

                         year  
authority_code year            
3.0            2005.0  2005.0  
               2005.0  2005.0  
               2002.0  2002.0  
               2005.0  2005.0  
               2005.0  2005.0  

[5 rows x 82 columns]

# corrected version: Never Touch

In [104]:
#correction version - more well matched with stata codes
#stata code
#xi: areg `o' fpsb_auction  i.year if `t'==1 & ctrl_exp_`t'==1 & post_experience>=`i' & pre_experience>=`i' & post_experience!=. & pre_experience!=. & missing==0, absorb(authority_code) vce(cluster auth_anno)

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['auth_anno', 'year'])

df_reg_exp['year'] = year
df_reg_exp['authority_code'] = id_auth

exog_var = ['fpsb_auction','year','authority_code']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
df_reg_exp.head()
#print(fe_res)

authority_code  fpsb_auction  n_bidders   discount  \
auth_anno year                                                        
26.0      2005.0              2           0.0       25.0  15.200000   
          2005.0              2           0.0       21.0  14.720000   
23.0      2002.0              2           0.0       72.0  17.879999   
26.0      2005.0              2           0.0      136.0  21.830000   
          2005.0              2           0.0       46.0  19.180000   

                  reserve_price work_category  complex_work  delay_ratio  \
auth_anno year                                                             
26.0      2005.0      1129754.0          OG02           3.0    40.547947   
          2005.0      1982000.0          OG01           2.0    17.362270   
23.0      2002.0       620000.0          OG03           1.0          NaN   
26.0      2005.0       738612.0          OG03           1.0     0.000000   
          2005.0       435225.0          OS28           2.0    38.333332   

                  overrun_ratio  days_to_award  ...  Iwork_ca_27  Iwork_ca_15  \
auth_anno year                                  ...                             
26.0      2005.0       4.843400            5.0  ...            0            0   
          2005.0       4.794236           50.0  ...            0            0   
23.0      2002.0      -0.004622            8.0  ...            0            0   
26.0      2005.0      -3.869733           62.0  ...            0            0   
          2005.0       4.913357           23.0  ...            0            0   

                  Iwork_ca_42  Iwork_ca_14  Iwork_ca_21  Iwork_ca_30  \
auth_anno year                                                         
26.0      2005.0            0            0            0            0   
          2005.0            0            0            0            0   
23.0      2002.0            0            0            0            0   
26.0      2005.0            0            0            0            0   
          2005.0            0            0            0            0   

                  Iwork_ca_16  Iwork_ca_5  Iwork_ca_43    year  
auth_anno year                                                  
26.0      2005.0            0           0            0  2005.0  
          2005.0            0           0            0  2005.0  
23.0      2002.0            0           0            0  2002.0  
26.0      2005.0            0           0            0  2005.0  
          2005.0            0           0            0  2005.0  

[5 rows x 82 columns]

# test

In [48]:
#iwork 유일하게 돌아가는 거
#PanelA_2
'''reg_col = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
for i in range(len(work_ca_order)):
    reg_col.append('Iwork_ca_'+str(i+1))

reg_col = reg_col + ['reserve_price', 'municipality']
print(reg_col)

df_reg_exp2 = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

x = df_reg_exp2[reg_col]
y = df_reg_exp2['discount']
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
model.summary()
'''

"reg_col = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']\nfor i in range(len(work_ca_order)):\n    reg_col.append('Iwork_ca_'+str(i+1))\n\nreg_col = reg_col + ['reserve_price', 'municipality']\nprint(reg_col)\n\ndf_reg_exp2 = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]\n\nx = df_reg_exp2[reg_col]\ny = df_reg_exp2['discount']\nmodel = sm.OLS(y,x).fit()\npredictions = model.predict(x)\nmodel.summary()\n"

In [84]:
#Iyeardelete
#version1
#Iyear빼고 독립변수 다 넣은 거: 유일하게 '돌아감' 값은 안 정확
reg_col = ['fpsb_auction']
for i in range(len(work_ca_order)):
    reg_col.append('Iwork_ca_'+str(i+1))

reg_col = reg_col + ['reserve_price', 'municipality']
print(reg_col)

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#year = pd.Categorical(df_reg_exp.year)
#df_reg_exp = df_reg_exp.set_index(['year'])
#df_reg_exp['year'] = year
df_reg_exp = df_reg_exp.dropna()

x = df_reg_exp[reg_col]
y = df_reg_exp['discount']
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
model.summary()


['fpsb_auction', 'Iwork_ca_1', 'Iwork_ca_2', 'Iwork_ca_3', 'Iwork_ca_4', 'Iwork_ca_5', 'Iwork_ca_6', 'Iwork_ca_7', 'Iwork_ca_8', 'Iwork_ca_9', 'Iwork_ca_10', 'Iwork_ca_11', 'Iwork_ca_12', 'Iwork_ca_13', 'Iwork_ca_14', 'Iwork_ca_15', 'Iwork_ca_16', 'Iwork_ca_17', 'Iwork_ca_18', 'Iwork_ca_19', 'Iwork_ca_20', 'Iwork_ca_21', 'Iwork_ca_22', 'Iwork_ca_23', 'Iwork_ca_24', 'Iwork_ca_25', 'Iwork_ca_26', 'Iwork_ca_27', 'Iwork_ca_28', 'Iwork_ca_29', 'Iwork_ca_30', 'Iwork_ca_31', 'Iwork_ca_32', 'Iwork_ca_33', 'Iwork_ca_34', 'Iwork_ca_35', 'Iwork_ca_36', 'Iwork_ca_37', 'Iwork_ca_38', 'Iwork_ca_39', 'Iwork_ca_40', 'Iwork_ca_41', 'Iwork_ca_42', 'reserve_price', 'municipality']


C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     6.153
Date:                Wed, 01 Jul 2020   Prob (F-statistic):            0.00322
Time:                        16:16:30   Log-Likelihood:                -217.58
No. Observations:                  86   AIC:                             441.2
Df Residuals:                      83   BIC:                             448.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction   1.107e-10   1.99e-11      5.573      0.000    7.12e-11     1.5e-10
Iwork_ca_1        9.8859      0.572     17.292      0.000       8.749      11.023
Iwork_ca_2    -1.331e-09   9.02e-11    -14.760      0.000   -1.51e-09   -1.15e-09
Iwork_ca_3             0          0        nan        nan           0           0
Iwork_ca_4             0          0        nan        nan           0           0
Iwork_ca_5             0          0        nan        nan           0           0
Iwork_ca_6             0          0        nan        nan           0           0
Iwork_ca_7             0          0        nan        nan           0           0
Iwork_ca_8             0          0        nan        nan           0           0
Iwork_ca_9             0          0        nan        nan           0           0
Iwork_ca_10            0          0        nan        nan           0           0
Iwork_ca_11            0          0        nan        nan           0           0
Iwork_ca_12            0          0        nan        nan           0           0
Iwork_ca_13            0          0        nan        nan           0           0
Iwork_ca_14            0          0        nan        nan           0           0
Iwork_ca_15            0          0        nan        nan           0           0
Iwork_ca_16            0          0        nan        nan           0           0
Iwork_ca_17            0          0        nan        nan           0           0
Iwork_ca_18            0          0        nan        nan           0           0
Iwork_ca_19            0          0        nan        nan           0           0
Iwork_ca_20            0          0        nan        nan           0           0
Iwork_ca_21            0          0        nan        nan           0           0
Iwork_ca_22            0          0        nan        nan           0           0
Iwork_ca_23            0          0        nan        nan           0           0
Iwork_ca_24            0          0        nan        nan           0           0
Iwork_ca_25            0          0        nan        nan           0           0
Iwork_ca_26            0          0        nan        nan           0           0
Iwork_ca_27            0          0        nan        nan           0           0
Iwork_ca_28            0          0        nan        nan           0           0
Iwork_ca_29            0          0        nan        nan           0           0
Iwork_ca_30            0          0        nan        nan           0           0
Iwork_ca_31            0          0        nan        nan           0           0
Iwork_ca_32            0          0        nan        nan           0           0
Iwork_ca_33            0          0        nan        nan           0           0
Iwork_ca_34            0          0        nan   

In [46]:
!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [79]:
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['year'])
#df_reg_exp['year'] = year
reg_col = ['fpsb_auction']
for i in range(len(work_ca_order)):
    reg_col.append('Iwork_ca_'+str(i+1))

#reg_col = reg_col + ['year','reserve_price', 'municipality(singular matrix)']

results = mt.reg(df_reg_exp, 'discount', ['fpsb_auction','reserve_price','year',''], fe_name = 'authority_code', cluster = 'auth_anno')
print(results)
#df_reg_exp.head()

Dependent variable:	discount
N:			1262
R-squared:		0.5034
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se     t   p>t CI_low CI_high
fpsb_auction  11.712 1.471 7.961 0.000  8.794  14.631
reserve_price  0.000 0.000 5.355 0.000  0.000   0.000
year           0.617 0.161 3.823 0.000  0.297   0.937



In [102]:
#Iyeardelete & work category index 
#iwork들어갈 때는 index하면 무조건 뒤짐
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['auth_anno', 'year'])
exog_var = ['fpsb_auction','year','authority_code']
#work_category = pd.Categorical(df_reg_exp.work_category)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','work_category'])
#df_reg_exp['work_category'] = work_category
#df_reg_exp = df_reg_exp.dropna()


x = df_reg_exp[['fpsb_auction','reserve_price','municipality']]
y = df_reg_exp['discount']
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               discount   R-squared (uncentered):                   0.728
Model:                            OLS   Adj. R-squared (uncentered):              0.727
Method:                 Least Squares   F-statistic:                              1123.
Date:                Wed, 01 Jul 2020   Prob (F-statistic):                        0.00
Time:                        16:50:12   Log-Likelihood:                         -4460.0
No. Observations:                1262   AIC:                                      8926.
Df Residuals:                    1259   BIC:                                      8941.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction     15.4347      0.940     16.424      0.000      13.591      17.278
reserve_price   6.18e-06   2.89e-07     21.383      0.000    5.61e-06    6.75e-06
municipality      7.3753      0.389     18.946      0.000       6.612       8.139
==============================================================================
Omnibus:                       30.352   Durbin-Watson:                   1.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.974
Skew:                           0.150   Prob(JB):                     7.00e-13
Kurtosis:                       3.987   Cond. No.                     4.37e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.37e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""